In [7]:
!pip install mediapipe --user


In [15]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face Mesh and drawing utilities
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Set up Face Mesh with appropriate parameters
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1, refine_landmarks=True,
    min_detection_confidence=0.5, min_tracking_confidence=0.5
)

# Define indices for eye landmarks
LEFT_EYE_INDICES = [133, 144, 145, 163, 157, 158, 160, 161]  # Left eye landmarks
RIGHT_EYE_INDICES = [362, 373, 374, 392, 386, 387, 389, 390]  # Right eye landmarks

def main():
    # Start video capture
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Failed to grab frame")
            break

        # Flip and convert the frame to RGB for MediaPipe processing
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to get facial landmarks
        results = face_mesh.process(rgb_frame)

        # Assume the user is not looking at the camera initially
        looking_straight = False

        if results.multi_face_landmarks:
            # Get the landmarks of the first detected face
            landmarks = results.multi_face_landmarks[0]

            ih, iw, _ = frame.shape  # Image dimensions

            # Calculate pupil positions by averaging the eye landmarks
            left_eye_x = sum(landmarks.landmark[i].x for i in LEFT_EYE_INDICES) / len(LEFT_EYE_INDICES)
            left_eye_y = sum(landmarks.landmark[i].y for i in LEFT_EYE_INDICES) / len(LEFT_EYE_INDICES)

            right_eye_x = sum(landmarks.landmark[i].x for i in RIGHT_EYE_INDICES) / len(RIGHT_EYE_INDICES)
            right_eye_y = sum(landmarks.landmark[i].y for i in RIGHT_EYE_INDICES) / len(RIGHT_EYE_INDICES)

            # Convert normalized landmarks to pixel coordinates
            left_pupil_pos = (int(left_eye_x * iw), int(left_eye_y * ih))
            right_pupil_pos = (int(right_eye_x * iw), int(right_eye_y * ih))

            # Draw circles around the pupils
            cv2.circle(frame, left_pupil_pos, 5, (0, 255, 0), -1)
            cv2.circle(frame, right_pupil_pos, 5, (0, 255, 0), -1)

            # Check if the user is looking at the camera based on pupil positions
            # Define thresholds
            horizontal_threshold = 0.1  # Maximum horizontal difference allowed
            vertical_threshold = 0.5  # Maximum vertical position allowed (looking straight)

            if (abs(left_eye_x - right_eye_x) < horizontal_threshold and 
                left_eye_y < vertical_threshold and right_eye_y < vertical_threshold):
                looking_straight = True

        # Display the alert if the user is not looking at the camera
        if not looking_straight:
            cv2.putText(frame, "Looking Straight", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255 ,0), 2)
        else:
            # Optionally, you can show a different message if they are looking straight
            cv2.putText(frame, "Please look at the camera!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 ,255), 2)

        # Show the video frame
        cv2.imshow('Eye Detection', frame)
        # Break the loop on 'q' key press
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
